In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('/content/intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

print("The Intent Data", data['intents'])
print()
print("Training Sentences: ", training_sentences)
print()
print("Labels: ", training_labels)
print()
print("Responses: ", responses)

The Intent Data [{'tag': 'google', 'patterns': ['google', 'search', 'internet'], 'responses': ['Redirecting to Google...']}, {'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Namaste', 'yo'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Get lost', 'Till next time', 'bbye'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you pro

In [3]:
print(num_classes)
print(len(data['intents']))

31
31


In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

training_labels

array([10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11,  9,  9,  9,  9,  9,
        9, 26, 26, 26, 26, 26, 21, 21, 21, 21, 21, 16, 16, 16,  0,  0,  7,
        7,  7,  7,  7,  7,  7, 29, 29, 29, 29, 29, 13, 13, 13, 13, 13, 22,
       22, 22, 15, 15, 15,  1,  1,  8,  8,  8,  8,  8, 28, 28, 28, 17, 17,
       17, 17,  4,  4,  4,  4,  3,  3,  3,  3, 19, 19, 20, 20, 18, 18, 18,
       14, 14, 14,  6,  6, 24, 24, 24, 24, 24, 12, 12, 12, 12, 12, 27,  5,
       25, 25, 25, 25, 25, 23, 23, 23,  2,  2,  2])

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 31)                527       
                                                                 
Total params: 17071 (66.68 KB)
Trainable params: 17071 (66.68 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [7]:
model.save("chat_model.h5")
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('label_encoder.pickle', 'rb') as ecn_file:
    lbl_encoder = pickle.load(ecn_file)
model = load_model("chat_model.h5")
def generate_response(user_input):
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, truncating='post', maxlen=20)
    predictions = model.predict(padded_sequence)
    class_index = np.argmax(predictions)
    tag = lbl_encoder.inverse_transform([class_index])[0]

    responses = data['intents']
    for intent in responses:
        if intent['tag'] == tag:
            result = np.random.choice(intent['responses'])
            break

    return result

In [9]:
response = generate_response("Hi there!")
print(response)

1/1 [==============================] - 0s 140ms/step
Hi there, how can I help?


In [10]:
import random
def chat():
    print("Start Talking with the bot(type quit to stop!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = generate_response(inp)

        print("Bot: ", results)

In [ ]:
chat()

Start Talking with the bot(type quit to stop!
You: hi
1/1 [==============================] - 0s 23ms/step
Bot:  Good to see you again
